<h2>Table comparing different possible Cl2-to-CH4 ratios</h2>

In [1]:
# Some imports to stop some annoying yellow underlines
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import pandas as pd
import importlib
import types

# Very important lines to run the setup notebooks!!!
%run do_plot_setup.ipynb
%run do_radical_count_setup.ipynb
get_cl_radical_budget = get_cl_radical_budget # Tautology to get rid of the yellow underlines
import cl2_utilities as cl2_utils

# Extract some shared values that are used to facilitate plotting. Call 'display' on any of them to see their contents.
plot_default_properties = get_and_apply_plot_defaults() #Ignore yellow underline
line_default_properties = get_default_line_properties() #Ignore yellow underline
cl2_experiment_constants = get_cl2_experiment_constants() #Ignore yellow underline
(all_conversion_data,all_bypass_data) = get_experimental_data() #Ignore underline. These two dataframes have all the experimental results.

In [2]:
# Load the data from the Cl2 sweep to use here

# Load data for the Cl2 sweep trial
current_data = types.SimpleNamespace()

# Select data from only the the experiments we want
exps = ["09-28-24 Vary Cl2 (20-90 ppm) at Fixed CH4","09-19-24 Vary Cl2 (5-30 ppm) at Fixed CH4"]
bypass_data = all_bypass_data[all_bypass_data["experiment_name"].isin(exps)]
conversion_data = all_conversion_data[all_conversion_data["experiment_name"].isin(exps)]
conversion_data = conversion_data[(conversion_data['experiment_name']=="09-19-24 Vary Cl2 (5-30 ppm) at Fixed CH4") | # Use exp. with 100 ppm Cl2 tank for Cl2 5-30 ppm and exp. with 1000 ppm Cl2 tank for 45-90 ppm.
                                  (conversion_data['Cl2 LabJack: Cl2 reading minus zero (mV) baseline']>40)] # Proxy for >=45 ppm is a Cl2 measurement of more than 85mV
conversion_data=conversion_data.sort_values(by='Cl2 LabJack: Cl2 reading minus zero (mV) baseline',ascending=True)
current_data.conversion_data = conversion_data
current_data.bypass_data = bypass_data

# The Cl2 tank ppms are set manually here based on notes taken during each experiment; it's not recorded automatically in software.
cl2_tank_ppms = np.array([1000 if b>40 else 100 for b in conversion_data['Cl2 LabJack: Cl2 reading minus zero (mV) baseline']]) # We've confirmed that in this subset of trials, only Cl2 conc's 45 and over used the 1000 ppm Cl2 tank

# Extract the values and 95% CI from the CH4 conversion data
current_data.ch4_baselines = conversion_data["Selected: CH4 (ppm) baseline"]
((current_data.ch4_conversions,current_data.ch4_95_ci),) = cl2_utils.extract_spectrometer_data_from_conversions(conversion_data,
            fields=['Selected: CH4 (ppm)'], percent_accuracy_95 = cl2_experiment_constants.picarro_percent_accuracy_95, absolute_accuracy_95 = cl2_experiment_constants.picarro_absolute_accuracy_95)

# Extract the values and 95% CI from the Cl2 conversion data
(current_data.cl2_baseline, current_data.cl2_conversions, current_data.cl2_conversion_95_ci) = cl2_utils.extract_cl2_data_from_conversions(conversion_data,bypass_data,cl2_tank_ppms,cl2_experiment_constants.cl2_mfc_sccm_accuracy_95,cl2_experiment_constants.cl2_node_absolute_accuracy_95)
current_data.inlet_cl2 = current_data.cl2_baseline

# Compute the Cl* efficiencies
(current_data.cl_budget,current_data.cl_budget_95_ci) = get_cl_radical_budget(current_data.inlet_cl2)
current_data.cl_per_ch4 = current_data.cl_budget / current_data.ch4_conversions
current_data.cl2_per_ch4_95_ci = cl2_utils.get_95_ci_of_ratio(current_data.cl_budget,current_data.cl_budget_95_ci,current_data.ch4_conversions,current_data.ch4_95_ci)

# Save to variable and report success
cl2_sweep_data = current_data
print("Loaded results for plotting into object 'cl2_sweep_data'.")

Loaded results for plotting into object 'cl2_sweep_data'.


In [13]:
from IPython.display import display, HTML
from pretty_html_table import build_table

#dummy_df = pd.DataFrame(np.random.randint(0,5,size=(5, 4)), columns=list('ABCD'))

inlet_CH4 = 30 # ppm
df = pd.DataFrame()
df['Inlet [Cl2] (ppm)'] = np.array(cl2_sweep_data.cl2_baseline)
df['Cl2-to-CH4 ratio'] = np.array(cl2_sweep_data.cl2_baseline)/inlet_CH4
df['CH4 conversion (ppm)'] = np.array(cl2_sweep_data.ch4_conversions)
df['CH4 conversion (%)'] = 100*np.array(cl2_sweep_data.ch4_conversions)/inlet_CH4

# Round the whole dataframe for formatting purposes
round_dict = {'Inlet [Cl2] (ppm)':0, 'Cl2-to-CH4 ratio':2, 'CH4 conversion (ppm)':1,
               'CH4 conversion (%)':0}
df = df.round(round_dict)
for k,v in round_dict.items():
    if v==0:
        df[k] = df[k].astype('int')

def do_subscripts(text):
    text = text.replace('CH4','CH<sub>4</sub>')
    text = text.replace('Cl2','Cl<sub>2</sub>')
    return text

from IPython.display import display, Markdown, Latex
display(Markdown(do_subscripts(df.to_markdown())))


|    |   Inlet [Cl<sub>2</sub>] (ppm) |   Cl<sub>2</sub>-to-CH<sub>4</sub> ratio |   CH<sub>4</sub> conversion (ppm) |   CH<sub>4</sub> conversion (%) |
|---:|--------------------:|-------------------:|-----------------------:|---------------------:|
|  0 |                  10 |               0.33 |                    5.7 |                   19 |
|  1 |                  20 |               0.67 |                   12.4 |                   41 |
|  2 |                  30 |               1    |                   18.6 |                   62 |
|  3 |                  45 |               1.5  |                   23.1 |                   77 |
|  4 |                  60 |               2    |                   28.1 |                   94 |
|  5 |                  90 |               3    |                   30   |                  100 |